In [ ]:
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from tqdm import tqdm
import sys

_ = load_dotenv(find_dotenv())  # read local .env file
sys.path.append(Path("..").resolve().as_posix())

In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

from src.core.index import Index, ID_KEY
from src.core.retrievers.self_query_multi_vector import build_retriever
from src.core.generator import build_generator

In [ ]:
# index = Index.from_metadata(
#     metadata_path=Path("../resources/doc_metadata.jsonl").resolve(),
#     lib_path=Path("../../docs_md").resolve(),
#     persist_path=Path("index_storage").resolve(),
#     embedding_function=AzureOpenAIEmbeddings(
#         azure_deployment="text-embedding-ada-002", api_version="2023-07-01-preview"
#     ),
# )

index = Index.from_persist_path(
    persist_path=Path("index_storage").resolve(),
    embedding_function=AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-ada-002", api_version="2023-07-01-preview"
    ),
)

In [ ]:
llm = AzureChatOpenAI(
    temperature=0.0,
    azure_deployment="gpt4o",
    openai_api_version="2023-07-01-preview",
    max_retries=0,
)

retriever = build_retriever(
    llm=llm, vectorstore=index.vectorstore, docstore=index.docstore, id_key=ID_KEY
)

In [ ]:
generator = build_generator(llm=llm, retriever=retriever)

In [ ]:
from langchain.globals import set_debug

set_debug(False)

response = generator.invoke(
    {"input": "Tell me basics of EdgeQL"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

In [ ]:
def pretty_print_response(response):
    print(f"Prompt: {response['input']}\n\n")
    print(f"Answer:\n{response['answer'].answer}\n\n")
    print("***************************************************************\n")
    for citation in response["answer"].citations:
        doc = response['context'][citation.source_id]
        print(f"Source {citation.source_id}: {doc.metadata['source']}\n")
        print(f"Category: {doc.metadata['category']}\n")
        print(f"Quote:\n{citation.quote}\n\n")

In [ ]:
pretty_print_response(response)